# Анализ функциональности

**Цель: исследовать поведение пользователей в обновленном приложении.**

### **Задачи:** 
- Понять, как пользователи взаимодействуют с продуктом, и соотнести идеальный путь пользователей с фактическим.
- Выявить этапы, которые занимают больше всего времени, с тем чтобы поработать над их улучшением.
- Проанализировать зависимость оплат от прохождения обучения.

### **Формализованные задачи:** 
- 1.Определить самые распространённые пути прохождения (последовательности) этапов в приложении.
- 2.Посмотреть на среднее время между различными этапами и выделить самые большие временные промежутки.
- 3.Определить, существует ли различие в частоте и средней величине оплат между тремя группами пользователей:
   - пользователями, которые прошли обучение хотя бы раз;
   - пользователями, которые начали обучение, но не прошли его ни разу;
   - пользователями, которые не начинали обучение, а сразу же перешли к выбору уровня сложности.

In [15]:
import pandas as pd

In [16]:
event = pd.read_csv('gd_quize/7_4_Events.csv',sep = ',')
purchase = pd.read_csv('gd_quize/purchase.csv',sep = ',')

In [17]:
display(event.head(5))
display(purchase.head(5))

,id,event_type,selected_level,start_time,tutorial_id,user_id
0,28903,registration,NaN,2016-05-11T23:40:55,NaN,12583
1,28904,registration,NaN,2016-05-11T23:49:58,NaN,12584
2,28905,registration,NaN,2016-05-12T00:53:07,NaN,12585
3,28906,tutorial_start,NaN,2016-05-12T01:32:20,17562.0,12585
4,28907,tutorial_finish,NaN,2016-05-12T01:34:53,17562.0,12585


,id,user_id,event_datetime,amount
0,15674,12584,2016-05-12T10:34:16,100
1,15675,12985,2016-05-13T08:25:56,50
2,15676,12828,2016-05-13T16:33:46,50
3,15677,12598,2016-05-14T01:09:37,150
4,15678,13037,2016-05-14T01:24:46,100


In [18]:
# выведем список пользователей зарегистрировавшихся в 2018 году
mask_year = (event['event_type'] == 'registration') & (event['start_time'] >= '2018-01-01') & (event['start_time'] < '2019-01-01')
registered_users = event[mask_year]['user_id'].values 
# применим фильтр и создадим новый датафрейм
events_registred = event[event['user_id'].isin(registered_users)] 
events_registred.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 66959 entries, 51405 to 118364
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   id              66959 non-null  int64  
 1   event_type      66959 non-null  object 
 2   selected_level  8342 non-null   object 
 3   start_time      66959 non-null  object 
 4   tutorial_id     32954 non-null  float64
 5   user_id         66959 non-null  int64  
dtypes: float64(1), int64(2), object(3)
memory usage: 3.6+ MB


In [19]:
purchase.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5956 entries, 0 to 5955
Data columns (total 4 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   id              5956 non-null   int64 
 1   user_id         5956 non-null   int64 
 2   event_datetime  5956 non-null   object
 3   amount          5956 non-null   int64 
dtypes: int64(3), object(1)
memory usage: 186.2+ KB


In [20]:
purchase_registred = purchase[purchase['user_id'].isin(registered_users)]
purchase_registred.info()
purchase_registred.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1600 entries, 1171 to 2778
Data columns (total 4 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   id              1600 non-null   int64 
 1   user_id         1600 non-null   int64 
 2   event_datetime  1600 non-null   object
 3   amount          1600 non-null   int64 
dtypes: int64(3), object(1)
memory usage: 62.5+ KB


,id,user_id,event_datetime,amount
1171,16845,27845,2018-01-03T18:53:43,100
1172,16846,27865,2018-01-04T14:46:10,250
1174,16848,27911,2018-01-07T08:19:12,50
1175,16849,27910,2018-01-07T12:11:34,100
1176,16850,27940,2018-01-07T13:16:41,200


Для того чтобы понимать, как пользователи переходят из этапа в этап и на каких из них возникают сложности, мы должны определить конверсию на каждом из этапов воронки. То есть нам нужно понять, какой процент пользователей переходит с предыдущего этапа на следующий.

**Изучаем событие registration**

In [21]:
# Сколько пользователей совершили событие registration
events_registred[events_registred['event_type']=='registration']['user_id'].nunique() 

19926

In [22]:
events_registred['user_id'].nunique()

19926

Все пользователи совершают событие registration. Это обусловлено тем, что данный этап является обязательным для выполнения и без него пользователь не сможет перейти дальше.

**Изучаем событие tutorial_start**

In [23]:
# Срез данных по событию tutorial_start
events_registred[events_registred['event_type']=='tutorial_start'].head(5) 

,id,event_type,selected_level,start_time,tutorial_id,user_id
51412,80315,tutorial_start,NaN,2018-01-01T14:54:40,31505.0,27836
51413,80316,tutorial_start,NaN,2018-01-01T15:00:51,31506.0,27835
51415,80318,tutorial_start,NaN,2018-01-01T15:40:43,31507.0,27836
51417,80320,tutorial_start,NaN,2018-01-01T17:47:40,31508.0,27833
51420,80323,tutorial_start,NaN,2018-01-01T19:11:36,31509.0,27839


Число пользователей прошедших регистрацию больше чем обучение

In [24]:
# Определим процент пользователей, которые перешли к выполнению обучения, и запишем его в переменную percent_tutorial_start_users.
registered_users_count = events_registred[events_registred['event_type']=='registration']['user_id'].nunique() 
tutorial_users_count = events_registred[events_registred['event_type']=='tutorial_start']['user_id'].nunique() 
percent_tutorial_start_users = tutorial_users_count / registered_users_count
print(f'Процент пользователей, начавших обучение, от общего числа зарегистрировавшихся: {percent_tutorial_start_users:.2%}')

Процент пользователей, начавших обучение, от общего числа зарегистрировавшихся: 59.51%


**Изучаем событие tutorial_finish**

In [25]:
# Какое количество пользователей проходит обучение до конца (событие tutorial_finish)
tutorial_finish_users_count = events_registred[events_registred['event_type']=='tutorial_finish']['user_id'].nunique()
# Рассчитаем процент пользователей, завершивших обучение, среди пользователей, которые начали обучение. 
tutorial_completion_rate = tutorial_finish_users_count / tutorial_users_count 
print(f'Процент пользователей, завершивших обучение: {tutorial_completion_rate:.2%}')

Процент пользователей, завершивших обучение: 86.44%


В приложении достаточно хороший процент прохождения обучения. Но есть куда стремиться для его увеличения.

**Изучаем событие level_choice**

Из логики нашего приложения следует, что выбор уровня сложности можно совершить и без прохождения обучения. Поэтому рассчитаем долю пользователей percent_level_choice_users, которые выбрали уровень сложности, от общего числа зарегистрировавшихся:

In [26]:
level_choice_users_count = events_registred[events_registred['event_type']=='level_choice']['user_id'].nunique() 
percent_level_choice_users = level_choice_users_count / registered_users_count
print(f'Процент пользователей, выбравших уровень сложности тренировок (от общего числа зарегистрировавшихся):\
    {percent_level_choice_users:.2%}')

Процент пользователей, выбравших уровень сложности тренировок (от общего числа зарегистрировавшихся):    41.86%


Меньше половины пользователей (41.76 %) доходят до этапа выбора уровня сложности вопросов. А ведь этот этап напрямую влияет на то, что пользователь будет пользоваться приложением через бесплатные возможности, которые в дальнейшем могут привести к оплате.


**Изучаем pack_choice** 

In [27]:
training_choice_user_count = events_registred[events_registred['event_type']=='pack_choice']['user_id'].nunique()
percent_training_choice_users = training_choice_user_count / level_choice_users_count 
print(f'Процент пользователей, выбравших пакет бесплатных вопросов (от числа пользователей, которые выбрали уровень сложности:\
    {percent_training_choice_users:.2%}')
print(level_choice_users_count)

Процент пользователей, выбравших пакет бесплатных вопросов (от числа пользователей, которые выбрали уровень сложности:    68.77%
8342


**Изучаем покупку платных пакетов**

In [28]:
# Рассчитаем процент пользователей, которые оплатили вопросы, от числа пользователей, которые приобрели бесплатные вопросы:
paying_users_count = purchase_registred['user_id'].nunique()
percent_of_paying_users = paying_users_count / training_choice_user_count
print(f'Процент пользователей, которые оплатили вопросы (от числа пользователей, которые выбрали тренировки):\
    {percent_of_paying_users:.2%}')
# Посмотрим, какой процент составляют покупатели от общего числа зарегистрировавшихся
purchase_rate = paying_users_count / registered_users_count 
print(f'Процент пользователей, которые оплатили вопросы(от числа зарегистрировавшихся пользователей):\
    {purchase_rate:.2%}')

Процент пользователей, которые оплатили вопросы (от числа пользователей, которые выбрали тренировки):    27.89%
Процент пользователей, которые оплатили вопросы(от числа зарегистрировавшихся пользователей):    8.03%


### АНАЛИЗ УНИКАЛЬНЫХ ПОЛЬЗОВАТЕЛЬСКИХ ПУТЕЙ

Наша воронка предсказуемая: почти все этапы идут последовательно друг за другом. Однако нужно иметь в виду один нюанс: есть пользователи, которые не начинают обучение, есть такие, кто его не заканчивает, а есть те, кто проходят его по несколько раз. Отследив уникальные пользовательские пути, мы можем визуально оценить, какие успешные пути существуют, а также заметить отклонения.

In [83]:
# Объединим датафрейм с событиями с датафреймом по оплатам
# Добавим событие purchase
purchase['event_type'] = 'purchase' 
purchase_registred['event_type'] = 'purchase'
# Переименуем столбцы с одинаковыми названиями
event = event.rename(columns={'id':'event_id'}) 
purchase = purchase.rename(columns={'id':'purchase_id'})
events_registred = events_registred.rename(columns={'id':'event_id'})
purchase_registred = purchase_registred.rename(columns={'id':'purchase_id'})
purchase_registred = purchase_registred.rename(columns={'event_datetime':'start_time'})
# Объединяем датафреймы
total_events = pd.concat([events_registred,purchase_registred],sort=False) 
total_events.head(10)
display(total_events['event_type'].value_counts())

registration       19926
tutorial_start     18050
tutorial_finish    14904
level_choice        8342
pack_choice         5737
purchase            1600
Name: event_type, dtype: int64

Теперь сбросим индексы объединённого датафрейма (так как после объединения они дублировались и несут мало смысла) с помощью метода reset_index() и отсортируем все события по возрастанию времени с помощью sort_values()

In [35]:
total_events = total_events.reset_index(drop=True).sort_values('start_time') # сформировали выборку, в которой объединили события и оплаты

Воссоздаём последовательность событий поведения пользователя

In [36]:
user_path = total_events.groupby(['user_id'])['event_type'].apply(list).reset_index()
user_path.head()

,user_id,event_type
0,27832,[registration]
1,27833,"[registration, tutorial_start, tutorial_finish]"
2,27834,"[registration, tutorial_start, tutorial_finish]"
3,27835,"[registration, tutorial_start, tutorial_finish..."
4,27836,"[registration, tutorial_start, tutorial_start,..."


Выявляем наиболее популярные пути.  Преобразуем список событий в строку event_path. Эта операция нужна для оптимизации скорости объединения.

In [87]:
user_path['event_path'] = user_path['event_type'].apply(lambda x: " > ".join(x))

In [86]:
# Cгруппируем датафрейм по столбцу event_path, подсчитав число пользователей
user_paths = user_path.groupby('event_path')['user_id'].nunique().sort_values(ascending=False)
# Выведем 10 самых популярных путей
user_paths.head(10)

event_path
registration                                                                                                       7970
registration > tutorial_start > tutorial_finish > level_choice > pack_choice                                       2796
registration > tutorial_start > tutorial_finish                                                                    1956
registration > tutorial_start > tutorial_finish > level_choice                                                     1713
registration > tutorial_start > tutorial_finish > level_choice > pack_choice > purchase                            1083
registration > tutorial_start                                                                                       842
registration > tutorial_start > level_choice > pack_choice                                                          346
registration > tutorial_start > tutorial_finish > tutorial_start > tutorial_finish                                  323
registration > tutorial_start

Только 1 из 10 самых популярных последовательностей привела к покупке

In [39]:
# Посмотрим, какие ещё последовательности содержат в себе оплату
user_paths[user_paths.index.str.contains('purchase')].head(10)

event_path
registration > tutorial_start > tutorial_finish > level_choice > pack_choice > purchase                                                                                                             1083
registration > tutorial_start > level_choice > pack_choice > purchase                                                                                                                                124
registration > tutorial_start > tutorial_finish > level_choice > pack_choice > tutorial_start > tutorial_finish > purchase                                                                           101
registration > tutorial_start > tutorial_finish > tutorial_start > tutorial_finish > level_choice > pack_choice > purchase                                                                            52
registration > tutorial_start > tutorial_start > tutorial_finish > level_choice > pack_choice > purchase                                                                                 

Большинство последовательностей, которые содержат в себе оплату, также содержат старт обучения. Это наводит на гипотезу, что вероятность оплаты зависит от того, **проходил ли пользователь обучение**.

### Анализ временных промежутков

Время между этапами может сообщить о вовлечённости пользователей в приложение, а также о роли отдельных этапов на развитие этой вовлечённости. Если окажется, что у пользователей, которые проходят обучение, время до выбора вопросов меньше, чем у тех, кто его не проходит, то разработчикам нужно будет придумывать способы, как вовлекать пользователя в прохождение данного этапа.

Определим время между собтиями регистрации и начала обучения

In [96]:
# Создадим отдельный df только регистрации
registration_df = total_events[total_events['event_type']=='registration'] 
# Проверим что на 1 пользрвателя приходится 1 регистрация
registration_df['user_id'].value_counts().mean() 

1.0

In [95]:
tutorial_df = total_events[total_events['event_type']=='tutorial_start'] # создадим отдельный df только обучения
tutorial_df['user_id'].value_counts().mean() # проверим сколько в среднем обучений проходит 1 пользователь

1.522179119581717

In [94]:
# Отсортируем и удалим дупликаты
tutorial_start_df_wo_duplicates = tutorial_df.sort_values('start_time').drop_duplicates('user_id')
# Будем проверять только 1-ое обучение
tutorial_start_df_wo_duplicates['user_id'].value_counts().mean() 

1.0

In [97]:
# Переименуем столбцы
registration_df = registration_df[['user_id','start_time']].rename(columns={'start_time':'registration_time'})
tutorial_start_df_wo_duplicates = tutorial_start_df_wo_duplicates[
    ['user_id','tutorial_id','start_time']
].rename(columns={'start_time':'tutorial_start_time'})
# Объединим таблицы по столбцу user_id 
merged_df = registration_df.merge(tutorial_start_df_wo_duplicates,on='user_id',how='inner')
merged_df['tutorial_start_time'] = pd.to_datetime(merged_df['tutorial_start_time'],yearfirst=True) 
merged_df['registration_time'] = pd.to_datetime(merged_df['registration_time'] ,yearfirst=True)
# Разница между временем начала обучения и временем регистрации
merged_df['timedelta'] = merged_df['tutorial_start_time'] - merged_df['registration_time']
merged_df.head()

,user_id,registration_time,tutorial_id,tutorial_start_time
0,27833,2018-01-01T04:07:25,31508.0,2018-01-01T17:47:40
1,27834,2018-01-01T08:35:10,31510.0,2018-01-01T19:46:11
2,27835,2018-01-01T11:54:47,31506.0,2018-01-01T15:00:51
3,27836,2018-01-01T13:28:07,31505.0,2018-01-01T14:54:40
4,27839,2018-01-01T18:24:01,31509.0,2018-01-01T19:11:36


In [48]:
# Среднее время, которое проходит между регистрацией пользователя и началом первого обучения
merged_df['timedelta'].mean() 

Timedelta('0 days 04:38:24.019817844')

In [49]:
# Проверим медианное и среднее значение
merged_df['timedelta'].describe() 

count                        11858
mean     0 days 04:38:24.019817844
std      0 days 04:15:09.650705034
min                0 days 00:00:34
25%         0 days 01:21:37.500000
50%         0 days 03:22:08.500000
75%         0 days 06:47:42.750000
max                1 days 16:03:46
Name: timedelta, dtype: object

Таким образом:
- четверть пользователей тратит меньше 1 часа 26 минут на переход от регистрации к началу обучения;
- половина всех пользователей тратит между регистрацией и началом обучения менее 3 часов 28 минут.

Такая сегментация пользователей может дать нам важные опорные точки для оптимизации времени. Мы можем сосредоточится не на оптимизации среднего времени (поскольку среднее будет увеличено за счёт пользователей, которые долго проходят этап), а на том, чтобы оптимизировать медианное время

Измеряем время обучения:

In [99]:
# Создадим df с событием окончания обучения
tutorial_finish_df = total_events[total_events['event_type']=='tutorial_finish']
# Возьмём время окончания первого обучения для тех обучений, которые были закончены
# Сформируем список с идентификаторами первых обучений
first_tutorial_ids = tutorial_start_df_wo_duplicates['tutorial_id'].unique()
# Отфильтруем датафрейм  и перезапишем
# Оставив в нём только события для таких обучений, которые были первыми для пользователя
tutorial_finish_df = tutorial_finish_df[tutorial_finish_df['tutorial_id'].isin(first_tutorial_ids)]
# Проверим, что на 1 пользователя приходится 1 обучение
tutorial_finish_df['user_id'].value_counts().mean()

1.0

In [53]:
tutorial_finish_df = tutorial_finish_df[['user_id','start_time']].rename(columns={'start_time':'tutorial_finish_time'})
# Оставим только необходимые столбцы
merged_df_2 = tutorial_start_df_wo_duplicates.merge(tutorial_finish_df,on='user_id',how='inner')
# Объединим df с началом и концом обучения
merged_df_2['tutorial_finish_time'] = pd.to_datetime(merged_df_2['tutorial_finish_time'],yearfirst=True)
merged_df_2['tutorial_start_time'] = pd.to_datetime(merged_df_2['tutorial_start_time'],yearfirst=True)
# Найдем дельту промежутка начала и конца обучения
merged_df_2['timedelta'] = merged_df_2['tutorial_finish_time'] - merged_df_2['tutorial_start_time'] 

In [54]:
display(merged_df_2.head())
print(merged_df_2['timedelta'].mean())
print(merged_df_2['timedelta'].describe())

,user_id,tutorial_id,tutorial_start_time,tutorial_finish_time,timedelta
0,27835,31506.0,2018-01-01 15:00:51,2018-01-01 15:06:15,0 days 00:05:24
1,27833,31508.0,2018-01-01 17:47:40,2018-01-01 17:50:08,0 days 00:02:28
2,27839,31509.0,2018-01-01 19:11:36,2018-01-01 19:16:32,0 days 00:04:56
3,27834,31510.0,2018-01-01 19:46:11,2018-01-01 19:48:01,0 days 00:01:50
4,27845,31512.0,2018-01-02 03:03:44,2018-01-02 03:06:48,0 days 00:03:04


0 days 00:03:53.174160732
count                         9830
mean     0 days 00:03:53.174160732
std      0 days 00:01:44.803217992
min                0 days 00:00:16
25%                0 days 00:02:28
50%                0 days 00:03:42
75%                0 days 00:05:08
max                0 days 00:10:06
Name: timedelta, dtype: object


75 % пользователей проходят обучение менее чем за 5 минут 3 секунды.

 Определяем время между registration и level_choice

In [55]:
level_choise_df = total_events[total_events['event_type']=='level_choice']
print(level_choise_df['user_id'].value_counts().mean())
# выберем необходимые столбц и переименуем
level_choise_df = level_choise_df[['user_id','start_time']
].rename(columns={'start_time':'level_choice_time'})
# присоедним таблицы 
merged_df_3 = level_choise_df.merge(registration_df,on='user_id',how='inner')
display(merged_df_3.head())

1.0


,user_id,level_choice_time,registration_time
0,27835,2018-01-01T20:37:22,2018-01-01T11:54:47
1,27839,2018-01-01T22:37:50,2018-01-01T18:24:01
2,27840,2018-01-02T05:18:42,2018-01-01T20:53:21
3,27845,2018-01-02T06:19:18,2018-01-02T01:35:56
4,27842,2018-01-02T08:46:03,2018-01-01T23:18:46


In [100]:
merged_df_3['level_choice_time'] = pd.to_datetime(merged_df_3['level_choice_time'],yearfirst=True)
merged_df_3['registration_time'] = pd.to_datetime(merged_df_3['registration_time'],yearfirst=True)
# Найдм временной промежуток между регистрацией и выбором сложности
merged_df_3['timedelta'] = merged_df_3['level_choice_time'] - merged_df_3['registration_time']
print(merged_df_3['timedelta'].mean())
print(merged_df_3['timedelta'].describe())


0 days 07:10:19.169863342
count                         8342
mean     0 days 07:10:19.169863342
std      0 days 04:33:51.164488800
min                0 days 00:08:15
25%         0 days 03:53:16.500000
50%                0 days 06:03:28
75%         0 days 09:34:58.500000
max                1 days 18:48:25
Name: timedelta, dtype: object


Определим сколько в среднем для всех пользователей проходит времени между событием выбора уровня сложности тренировки до события выбора набора бесплатных вопросов.

In [102]:
pack_choice_df = total_events[total_events['event_type']=='pack_choice']
pack_choice_df.head()

,event_id,event_type,selected_level,start_time,tutorial_id,user_id,purchase_id,amount
51425,80328.0,pack_choice,NaN,2018-01-01T20:38:43,NaN,27835,NaN,NaN
51429,80332.0,pack_choice,NaN,2018-01-01T22:42:54,NaN,27839,NaN,NaN
51444,80347.0,pack_choice,NaN,2018-01-02T06:25:12,NaN,27845,NaN,NaN
51458,80361.0,pack_choice,NaN,2018-01-02T11:59:26,NaN,27849,NaN,NaN
51463,80366.0,pack_choice,NaN,2018-01-02T14:14:51,NaN,27843,NaN,NaN


In [103]:
pack_choice_df = pack_choice_df[['user_id','start_time']].rename(columns={'start_time':'pack_choice_time'})
merged_df_4 = level_choise_df.merge(pack_choice_df,on='user_id',how='inner')
display(merged_df_4.head())
print(merged_df_4['user_id'].value_counts().mean())
merged_df_4['level_choice_time'] = pd.to_datetime(merged_df_4['level_choice_time'],yearfirst=True)
merged_df_4['pack_choice_time'] = pd.to_datetime(merged_df_4['pack_choice_time'],yearfirst=True)


,user_id,level_choice_time,pack_choice_time
0,27835,2018-01-01T20:37:22,2018-01-01T20:38:43
1,27839,2018-01-01T22:37:50,2018-01-01T22:42:54
2,27845,2018-01-02T06:19:18,2018-01-02T06:25:12
3,27849,2018-01-02T11:53:11,2018-01-02T11:59:26
4,27843,2018-01-02T14:09:58,2018-01-02T14:14:51


1.0


In [62]:
merged_df_4['timedelta'] = merged_df_4['pack_choice_time'] - merged_df_4['level_choice_time']
print(merged_df_4['timedelta'].mean())
print(merged_df_4['timedelta'].describe())

0 days 00:05:17.128464354
count                         5737
mean     0 days 00:05:17.128464354
std      0 days 00:02:43.923107804
min                0 days 00:00:19
25%                0 days 00:03:05
50%                0 days 00:04:57
75%                0 days 00:07:08
max                0 days 00:15:48
Name: timedelta, dtype: object


Меньше 5 минут в среднем для всех пользователей проходит времени между событием выбора уровня сложности тренировки до события выбора набора бесплатных вопросов

Определим сколько в среднем для всех пользователей проходит времени между событием выбора бесплатных вопросов и первой оплатой.

In [104]:
purchase_time_df = total_events[total_events['event_type']=='purchase']
purchase_time_df.head()


,event_id,event_type,selected_level,start_time,tutorial_id,user_id,purchase_id,amount
1171,NaN,purchase,NaN,2018-01-03T18:53:43,NaN,27845,16845.0,100.0
1172,NaN,purchase,NaN,2018-01-04T14:46:10,NaN,27865,16846.0,250.0
1174,NaN,purchase,NaN,2018-01-07T08:19:12,NaN,27911,16848.0,50.0
1175,NaN,purchase,NaN,2018-01-07T12:11:34,NaN,27910,16849.0,100.0
1176,NaN,purchase,NaN,2018-01-07T13:16:41,NaN,27940,16850.0,200.0


In [105]:
purchase_time_df = purchase_time_df[['user_id','start_time']].rename(columns={'start_time':'purchase_time'})
merged_df_5 = pack_choice_df.merge(purchase_time_df,on='user_id',how='inner')
display(merged_df_5.head())
print(merged_df_5['user_id'].value_counts().mean())
merged_df_5['purchase_time'] = pd.to_datetime(merged_df_5['purchase_time'],yearfirst=True)
merged_df_5['pack_choice_time'] = pd.to_datetime(merged_df_5['pack_choice_time'],yearfirst=True)
merged_df_5['timedelta'] = merged_df_5['purchase_time'] - merged_df_5['pack_choice_time']
print(merged_df_5['timedelta'].mean())
print(merged_df_5['timedelta'].describe())

,user_id,pack_choice_time,purchase_time
0,27845,2018-01-02T06:25:12,2018-01-03T18:53:43
1,27865,2018-01-04T06:03:20,2018-01-04T14:46:10
2,27884,2018-01-04T16:22:03,2018-01-08T19:37:34
3,27910,2018-01-05T12:05:28,2018-01-07T12:11:34
4,27911,2018-01-05T17:40:37,2018-01-07T08:19:12


1.0
3 days 17:46:53.403125
count                         1600
mean        3 days 17:46:53.403125
std      2 days 04:37:20.225124289
min                0 days 00:44:50
25%         1 days 21:24:13.250000
50%         3 days 12:51:25.500000
75%         5 days 09:42:13.750000
max               10 days 18:33:59
Name: timedelta, dtype: object


3,5 дня в среднем для всех пользователей проходит между событием выбора бесплатных вопросов и первой оплатой.

## Проверка аналитической гипотезы.
Определим, существует ли различие в частоте и средней величине оплат между тремя группами пользователей

In [67]:
users_with_finished_tutorial = total_events[total_events['event_type']=='tutorial_finish']['user_id'].unique()
# Пользователи закончившие обучение хотя бы 1 раз
print(len(users_with_finished_tutorial)) 

10250


In [106]:
users_with_started_tutorial = total_events[total_events['event_type']=='tutorial_start']['user_id'].unique()
set_users_with_started_tutorial = set(users_with_started_tutorial)
set_users_not_finished_but_started_tutorial = set_users_with_started_tutorial.difference(set(users_with_finished_tutorial))
# Пользователи начавшие, но не закончившие обучение
print(len(set_users_not_finished_but_started_tutorial))

1608


In [69]:
# Ищем пользователей, сразу выбравших уровень сложности
all_users = total_events['user_id'].unique()
set_all_users = set(all_users)
set_users_not_started_tutorial = set_all_users.difference(set_users_with_started_tutorial)
print(len(set_users_not_started_tutorial))

8068


In [70]:
len(set_users_not_finished_but_started_tutorial) + len(set_users_not_started_tutorial)\
    + len(set(users_with_finished_tutorial))==len(set_all_users)

True

Считаем пользователей, которые завершили обучение и совершили оплату

In [71]:
purchase_finished_tutorial = purchase_registred[purchase_registred['user_id'].isin(users_with_finished_tutorial)]
# Кол-во пользователей закончивших обучение и оплативших
purchase_finished_tutorial['user_id'].nunique() 

1447

In [72]:
percent_of_purchase_1 = purchase_finished_tutorial['user_id'].nunique() / len(users_with_finished_tutorial)
print(f'Процент пользователей, которые оплатили тренировки (от числа пользователей, завершивших обучение):\
    {percent_of_purchase_1:.2%}')

Процент пользователей, которые оплатили тренировки (от числа пользователей, завершивших обучение):    14.12%


In [74]:
purchase_started_not_finished_tutorial = \
    purchase_registred[purchase_registred['user_id'].isin(set_users_not_finished_but_started_tutorial)]
print(purchase_started_not_finished_tutorial['user_id'].nunique())
percent_of_purchase_2 = purchase_started_not_finished_tutorial['user_id'].nunique() / len(set_users_not_finished_but_started_tutorial)
print(f'"Процент пользователей, которые оплатили тренировки (от числа пользователей, начавших обучение, но не завершивших):\
    {percent_of_purchase_2:.2%}')

131
"Процент пользователей, которые оплатили тренировки (от числа пользователей, начавших обучение, но не завершивших):    8.15%


In [75]:
purchase_started_not_finished_tutorial['amount'].mean()

104.9618320610687

In [76]:
purchase_not_started_tutorial = \
    purchase_registred[purchase_registred['user_id'].isin(set_users_not_started_tutorial)]
print(purchase_not_started_tutorial['user_id'].nunique()) 
percent_of_purchase_3 = purchase_not_started_tutorial['user_id'].nunique() / len(set_users_not_started_tutorial)
print(f'"Процент пользователей, которые оплатили тренировки (от числа пользователей, не начавших обучение):\
    {percent_of_purchase_3:.2%}')


22
"Процент пользователей, которые оплатили тренировки (от числа пользователей, не начавших обучение):    0.27%


In [77]:
purchase_not_started_tutorial['amount'].mean()

128.4090909090909

Как мы видим, процент пользователей, которые завершили обучение и совершили оплату, выше, чем процент пользователей, которые не начали обучение или не закончили его.

Это говорит о том, что факт прохождения обучения влияет на дальнейшую мотивацию использовать приложение. Существенной разницы в среднем чеке при этом не наблюдается, то есть обучение увеличивает вовлечённость, но выбор количества платных пакетов зависит индивидуально от пользователя